In [1]:
#https://gorelick.medium.com/fast-er-downloads-a2abd512aa26

import ee
import requests
import os
import shutil
import tqdm.notebook as tqdm

  
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com',project='ee-seasonal-snow-cover')


In [2]:
import rioxarray as rio
import pyproj 
popgrid = rio.open_rasterio('pop2021.tif',band_as_variable=True).band_1
crs='EPSG:3035' #popgrid.rio.crs
transformer = pyproj.transformer.Transformer.from_crs(crs,"epsg:4326", always_xy=True)

In [3]:
import numpy as np
np.random.seed(0)
if False:
    N=30
    Nlarge=100000
    x = np.random.randint(0, popgrid.shape[1], Nlarge)
    y = np.random.randint(0, popgrid.shape[0], Nlarge)

    p = popgrid.data[y,x]
    ix = np.where(p>2000)[0]
    ix = ix[:N]

    #p=p[y[ix],x[ix]]
    x=popgrid.x.data[x[ix]]
    y=popgrid.y.data[y[ix]]
    p = p[ix]

    points = [[float(x),float(y),int(p)] for x,y,p in zip(x,y,p)]


In [4]:
x,y = np.meshgrid(range(popgrid.shape[1]),range(popgrid.shape[0]))
x = x.flatten()
y = y.flatten()
ix = np.where(popgrid.data.flatten()>0)[0]
cump = np.cumsum(popgrid.data.flatten()[ix])
x = x[ix]
y = y[ix]
person = np.random.rand(100)*cump[-1]
ix = np.ceil(np.interp(person,cump,np.arange(len(cump)))).astype(int)
p = popgrid.data[y[ix],x[ix]]
x=popgrid.x.data[x[ix]]
y=popgrid.y.data[y[ix]]


points = [[float(x),float(y),int(p)] for x,y,p in zip(x.tolist(),y.tolist(),p.tolist())]
points

[[3911500.0, 2883500.0, 852],
 [4775500.0, 3101500.0, 8337],
 [4321500.0, 2942500.0, 528],
 [4895500.0, 2879500.0, 6360],
 [4239500.0, 2702500.0, 27],
 [4967500.0, 3014500.0, 1593],
 [4223500.0, 2728500.0, 980],
 [3061500.0, 3387500.0, 1828],
 [4058500.0, 4019500.0, 15],
 [5264500.0, 2606500.0, 113],
 [3961500.0, 3203500.0, 3974],
 [4450500.0, 2856500.0, 2273],
 [3751500.0, 2897500.0, 4372],
 [4432500.0, 3568500.0, 2650],
 [3691500.0, 1864500.0, 2515],
 [2665500.0, 1945500.0, 5424],
 [3029500.0, 1639500.0, 18859],
 [3971500.0, 3260500.0, 9366],
 [4208500.0, 3177500.0, 1608],
 [4229500.0, 3325500.0, 3441],
 [5159500.0, 4127500.0, 11688],
 [4882500.0, 3213500.0, 1169],
 [5285500.0, 2763500.0, 1823],
 [4199500.0, 3182500.0, 1931],
 [3159500.0, 2023500.0, 16515],
 [4733500.0, 3006500.0, 6708],
 [3664500.0, 2064500.0, 48625],
 [4286500.0, 3705500.0, 1885],
 [4269500.0, 2847500.0, 7446],
 [4353500.0, 2686500.0, 297],
 [4421500.0, 2384500.0, 1797],
 [3979500.0, 3171500.0, 9680],
 [4819500.0, 

In [5]:
from retry import retry

folder=os.path.abspath('tiles/')

#@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
  
  """Handle the HTTP requests to download an image."""

  # Generate the desired image from the given point.
  #filename = f'{folder}/{point[0]/1000:.0f}_{point[1]/1000:.0f}_pop{point[2]:.0f}.tif'

  pixelsize = 20 #meter
  tilewidth = 5000 #meter
  point[0] = np.floor(point[0]/tilewidth)*tilewidth
  point[1] = np.floor(point[1]/tilewidth)*tilewidth
  filename = f'{folder}/{int(tilewidth/1000)}km_{int(point[0]/1000)}_{int(point[1]/1000)}.tif'
  if os.path.exists(filename):
    print("Already exists: ", filename)
    return
  
  Npixels = int(tilewidth/pixelsize)
  corner_w = point[0] # + pixelsize/2
  corner_s = point[1] # + pixelsize/2
  corner_e = corner_w + tilewidth #- pixelsize
  corner_n = corner_s + tilewidth #- pixelsize
  region = ee.Geometry.Rectangle([
    ee.Geometry.Point((corner_w, corner_s),proj=crs),  
    ee.Geometry.Point((corner_e, corner_n),proj=crs)
  ], proj=crs, evenOdd=False)
  
  image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') #('LANDSAT/LC08/C02/T1')
           .filterBounds(region)
           .filterDate('2019', '2023') #more years to get more stable stats
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
           .filter(ee.Filter.calendarRange(6,10,'month')) #get rid of snowy season (and reduce memory usage)
           .median()
           .select('B1','B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12')
           )

  s1vv = (ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterDate('2021', '2022')
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    .select(['VV'])
    .median())

  s1vh = (ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterDate('2021', '2022')
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    .select(['VH'])
    .median())

  nightlight = (ee.ImageCollection('NASA/VIIRS/002/VNP46A2')
    .filterDate('2021', '2022')
    .select('DNB_BRDF_Corrected_NTL')
    .median())

  image = image.addBands([nightlight,s1vv,s1vh])

  # Fetch the URL from which to download the image.
  url = image.clip(region).getDownloadURL({
      'region': region,
      'crs':crs,
      'dimensions': f'{Npixels}x{Npixels}',
      'format': 'GEO_TIFF'})

  # Handle downloading the actual pixels.
  r = requests.get(url, stream=True)
  if r.status_code != 200:
    print("Error: ", index, point, r.text)
    raise r.raise_for_status()

  with open(filename, 'wb') as out_file:
    shutil.copyfileobj(r.raw, out_file)
  print("Done: ", index, filename) 

In [ ]:
getResult(0, points[0])

In [ ]:
for j in tqdm.tqdm(range(len(points))): #optional:use multiprocessing instead
    getResult(j, points[j])

  0%|          | 0/50 [00:00<?, ?it/s]

Already exists:  c:\Users\ag\Documents\Python\pop_europe\tiles/5km_4510_2335.tif
Done:  1 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_5305_1930.tif
Done:  2 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_4345_4465.tif
Done:  3 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_4305_3535.tif
Done:  4 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_3865_2610.tif
Done:  5 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_4970_3005.tif
Done:  6 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_4825_3060.tif
Done:  7 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_4105_3235.tif
Done:  8 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_3425_2370.tif
Done:  9 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_3160_2035.tif
Done:  10 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_5315_2730.tif
Done:  11 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_5605_1500.tif
Done:  12 c:\Users\ag\Documents\Python\pop_europe\tiles/5km_4085_2655.tif
Done:  13 c:\Users\ag\Documents\Python\p